In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2 as cv
from skimage.io import imread, imsave
import numpy as np
import PIL

In [3]:
def json_reader(json_location):
    """
    Read json file and return dict object in convenient for us format
    """
    with open(json_location) as a_f:
        jn = json.load(a_f)
    return jn 

def mask_generator(mask_json, path_to_save, img_height=3840, img_width=5760):
    """
    Generate mask(png image) from a give json description and save to a given path. 
    
    Example:
    
        for i in os.listdir(json_masks_path)[:-1]:
            json_mask = json_reader(json_masks_path + i) # walk 
            mask_generator(json_mask,path_to_save)

    """
    regions = mask_json['regions']
    
    img = np.zeros([img_height, img_width, 3],dtype=np.uint8)    
    for region in regions:
        curr_shape_attributes = region['shape_attributes']
        all_x_points = curr_shape_attributes['all_points_x']
        all_y_points = curr_shape_attributes['all_points_y']
        
        rr, cc = skimage.draw.polygon(all_y_points,
                                          all_x_points)
        
        '''
        cut is the attribute which I gave while segmenting. If cut is 0 - it is the tissue,
        and 1 - we should cut the tissue. It was done because of limitation of the segmentation
        tool.
        '''
        is_cut = int(region['region_attributes']['cut'])
        if not is_cut:
            img[rr,cc] = 255
        else:
            img[rr,cc] = 0 

    imsave(os.path.join(path_to_save, mask_json['filename']), img)

def concat_images(imga, imgb):
    """
    Combines two color image ndarrays side-by-side.
    """
    ha,wa = imga.shape[:2]
    hb,wb = imgb.shape[:2]
    max_height = np.max([ha, hb])
    total_width = wa+wb
    new_img = np.zeros(shape=(max_height, total_width, 3))
    new_img[:ha,:wa]=imga
    new_img[:hb,wa:wa+wb]=imgb
    return new_img

#     print(imgs_comb.shape)
#     plt.imshow(imgs_comb)
# combine_2_images_horizontally(imgs[0], masks[0])

In [4]:
JSON_MASKS_PATH = '/home/bohdan/histologie/data/masksJSON/'
PATH_TO_SAVE = '/home/bohdan/histologie/data/masks/'
MASKS_PATH = '/home/bohdan/histologie/data/masks/'
IMG_PATH = '/home/bohdan/histologie/data/images/'
list_im = os.listdir(IMG_PATH)

### Data reading

In [5]:
imgs = [PIL.Image.open(os.path.join(IMG_PATH,i)) for  i in list_im]
masks = [PIL.Image.open(os.path.join(MASKS_PATH,i)) for i in list_im]

### Concatenating images

In [6]:
for i in range(len(imgs)):
    comb_img = concat_images(np.asarray(imgs[i]), np.asarray(masks[i]))
    imsave(os.path.join('/home/bohdan/histologie/data/combinedImages', list_im[i]),comb_img / comb_img.max())

/home/bohdan/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


### Merging images

In [ ]:
for i in range(len(imgs)):
    comb_img = concat_images(np.asarray(imgs[i]), np.asarray(masks[i]))
    merged_img = cv.addWeighted(np.asarray(imgs[i]), 1, np.asarray(masks[i]), 0.3, 1)    
    imsave(os.path.join('/home/bohdan/histologie/data/mergedImages', list_im[i]),merged_img)